## Feature Selection and Data Cleaning

### Goals
The goal of this notebook is to provide functions to:
- clean data
- remove unneccessary/undesired features
- create dummy variable
- provide basis for feature engineering

---

### Variables to work with based on flights_test information <br>
**fl_date**: Flight Date (yyyy-mm-dd)
- split into 2 columns:
   - fl_month
    - fl_day_of_week

**mkt_unique_carrier**: Unique Marketing Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- combine with op_unique_carrier
    - Label Encode

**branded_code_share**: Reporting Carrier Operated or Branded Code Share Partners
- Detemine is or is not "codeshare"
- codeshare = 1
- not codeshare = 0

**mkt_carrier**: Code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. For analysis, use the Unique Carrier Code.
- drop this information

**mkt_carrier_fl_num**: Flight Number
- determine flight type

**op_unique_carrier**: Unique Scheduled Operating Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users,for example, PA, PA(1), PA(2). Use this field for analysis across a range of years.
- combine with mkt_unique_carrier
    - Label Encode

**tail_num**: Tail Number
- drop this value

**op_carrier_fl_num**: Flight Number
- drop this value

**origin_airport_id**: Origin Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- drop this value

**origin**: Origin Airport
- Label Encode

**origin_city_name**: Origin Airport, City Name
- plan to drop this column
- option to create a state variable from this as indicated by a a two letter abreviation 
    - move as dummy

**dest_airport_id**: Destination Airport, Airport ID. An identification number assigned by US DOT to identify a unique airport. Use this field for airport analysis across a range of years because an airport can change its airport code and airport codes can be reused.
- drop this value

**dest**: Destination Airport
- use dummy variables

**dest_city_name**: Destination Airport, City Name
- plan to drop this column
- option to create a state variable from this as indicated by a a two letter abreviation 
    - move as dummy

**crs_dep_time**: CRS Departure Time (local time: hhmm)
- create binned by hour

**crs_arr_time**: CRS Arrival Time (local time: hhmm)
- create binned by hour

**dup**: Duplicate flag marked Y if the flight is swapped based on Form-3A data
- in the 5,000,000 sample file we do not have any Y
- we will check for any rows containing Y in full dataset -- drop rows if present
- drop entire column
    - Technical info from [link](https://www.bts.gov/sites/bts.dot.gov/files/docs/explore-topics-and-geography/topics/airlines-and-airports/207741/technical-directive-no-27-time-2018.pdf) states - Form 3A:  On-Time Performance Data for Codeshare Flights (Long)
    - assumed to be an infrequent form

**crs_elapsed_time**: CRS Elapsed Time of Flight, in Minutes
- keep as is

**flights**: Number of Flights
- in the 5,000,000 sample file all values are equal to 1
- we will check for any rows containing different values in full dataset -- drop rows if present
- drop entire column

**distance**: Distance between airports (miles)
- keep as is
    - option to bin

---

#### Package Imports

In [1]:
import pandas as pd
import numpy as np
import datetime

In [5]:
import warnings
warnings.filterwarnings('ignore')

---

### Function Definition

*binning into months*

In [2]:
def get_month(date):
    """Returns the month the flight occurs
    INPUT date in form YYYY-MM-DD
    RETURNS month"""
    DATE = datetime.datetime.strptime(date, "%Y-%m-%d")
    return DATE.month

*binning into day of week*

In [42]:
def get_day_of_week(date):
    """Returns the day of the week
    INPUT date in the form YYYY-MM-DD
    RETURNS number of day of the week:
            where 0 = Monday and 6 = Sunday"""
    DATE = datetime.datetime.strptime(date, "%Y-%m-%d")
    return DATE.weekday()

*checking if codeshare*

In [4]:
def check_codeshare(branded_codeshare):
    words = branded_codeshare.split("_")
    if len(words) == 2:
        return 1
    return 0

*binning into hours*

In [5]:
def check_time_block(time):
    """Determine the hour of the day
    INPUT time HHMM
    RETURNS hour 0-23 where 0 == 12am and 23 == 11pm"""
    if type(time) == 'numpy.ndarray':
        time = time.astype(int)
    time = int(time)
    
    t = time // 100
    if t == 0:
        return 0
    return t-1

*binning into haul length* (not used)

In [6]:
def define_haul_length(distance):
    """Determine the haul length of a given distance
    INPUT distance (in miles)
    RETURNS {0: short haul, 1: medium haul, 2: long haul}"""
    
    if distance < 700:
        return 0 
    elif distance < 3000:
        return 1
    else: 
        return 2

**NEW FEATURE** get unique mkt_op_combo
> must apply dummy variables also

In [7]:
def mkt_op_combo(mkt_unique, op_unique):
    """Output a string for the unique combination of the mkt_unique and op_unique
        INPUT strings mkt_unique and op_unique
        RETURNS string of unique combo"""
    combo = mkt_unique+op_unique
    return combo

**NEW FEATURE** Travel in state or out of state

In [8]:
def get_staterelation(origin_city,dep_city):
    """Takes in a strings representing origin and destination and returns whether 
        the flight is within a single state or between states
        INPUT city (string format CITY, SS)
        RETURN SS"""
    or_info = origin_city.split(",")
    dep_info = dep_city.split(",")
    if or_info[1][1:] == dep_info[1][1:]:
        return 0
    else:
        return 1

**NEW FEATURE** Flight type

In [69]:
def flight_type(fl_num):
    """Takes in a fl_num and returns the type of flight
        INPUT fl_num
        RETURNs flight_type 
                2: premium, 1 : regular, 0 : regional, ferry, codeshare"""
    fl_num = str(fl_num)
    if len(fl_num) < 3:
        return 2 #preimum
    if len(fl_num) == 3:
        return 1 #reg
    else: 
        return 0 #regional affiliate, ferry, codeshare, etc.

---

### Applying Above

**Step 1**: Remove outliers (any -150 < arr_delay < 150) <br>
**Step 2**: Check 'flights' and 'dup' for unexpected values <br>
**Step 3**: Drop Columns ('mkt_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id', 'dup', 'flights') <br>
**Step 4**: Get Columns 'fl_month' and 'fl_day_of_week' from 'fl_date' <br>
$\;\;\;\;\;\;$*Step 4a*: Drop 'fl_date'<br>
**Step 5**: Use 'mkt_unique_carrier' and 'op_unique_carrier' to obtain 'mkt_op_combo' <br>
$\;\;\;\;\;\;$*Step 5a*: Drop 'mkt_unique_carrier' and 'op_unique_carrier' <br>
**Step 6**: Binarize 'branded_code_share' <br>
**Step 7**: Replace 'mkt_carrier_fl_num' with flight_type 'A-C' <br>
**Step 8**: Replace 'origin_city_name' and 'dest_city_name' with state_travel_type <br>
$\;\;\;\;\;\;$*Step 8a*: Drop 'origin_city_name' and 'dest_city_name' <br>
**Step 9**: Replace 'crs_dep_time' with hour <br>
**Step 10**: Replace 'crs_arr_time' with hour <br>
**Step 11** Get cat.codes for ('mkt_op_combo', 'origin_airport', 'dest_airport') <br>
$\;\;\;\;\;\;$*Step 11a* drop original columns for ('mkt_op_combo', 'origin_airport', 'dest_airport')

***Special Notes for full flights database***
- Also need to drop the following: <br>
'dep_time', 'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'arr_time', 'cancellation_code', 'diverted', 'actual_elapsed_time', 'first_dep_time', 'total_add_gtime', 'longest_add_gtime'

---

### Small DataFrame

In [22]:
df = pd.read_csv("flight_sample_small.csv")

In [15]:
df.head(1)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime
0,2019-01-11,AS,AS,AS,64,AS,N611AS,64,15841,WRG,...,1.0,82.0,0.0,0.0,0.0,0.0,0.0,1620.0,0.0,0.0


*Pre* <br>
$\;\;\;\;\;\;$dropping unneeded columns

In [24]:
df.drop(columns=['dep_time', 'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in','arr_time','cancellation_code', 'diverted', 'actual_elapsed_time', 'first_dep_time', 'total_add_gtime', 'longest_add_gtime'], inplace = True)

*Step 1*

In [25]:
df = df[df.arr_delay >= -225]
df = df[df.arr_delay <= 225]

*Step 2*

In [28]:
df.flights.unique()

array([1.])

In [29]:
df.dup.unique()

array(['N'], dtype=object)

*Step 3*

In [31]:
df.drop(columns = ['mkt_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id', 'dup', 'flights'], inplace = True)

*Step 4*

In [32]:
#fl_month
df['fl_month'] = df['fl_date'].map(get_month)

In [34]:
#fl_day_of_month
df['fl_day_of_week'] = df['fl_date'].map(get_day_of_week)

In [37]:
#drop_fl_date
df.drop(columns = 'fl_date', inplace=True)

*Step 5*

In [38]:
#get mkt_op_combo
df["mkt_op_combo"] = df.apply(lambda x: mkt_op_combo(x.mkt_unique_carrier, x.op_unique_carrier), axis=1)

In [41]:
df.drop(columns = ['mkt_unique_carrier', 'op_unique_carrier'], inplace = True)

*Step 6*

In [42]:
df["branded_code_share"] = df["branded_code_share"].apply(check_codeshare)

*Step 7*

In [44]:
#get fl_type
df["fl_type"] = df["mkt_carrier_fl_num"].apply(flight_type)

In [47]:
df.drop(columns = 'mkt_carrier_fl_num', inplace=True)

*Step 8*

In [48]:
df['state_travel_type'] = df.apply(lambda x: get_staterelation(x.origin_city_name, x.dest_city_name), axis=1)

In [51]:
df.drop(columns = ['origin_city_name','dest_city_name'], inplace=True)

*Step 9*

In [52]:
df['crs_dep_time'] = df['crs_dep_time'].apply(check_time_block)

*Step 10*

In [54]:
df['crs_arr_time'] = df['crs_arr_time'].apply(check_time_block)

*Step 11* <br> for large data set revised to use label encoding

In [60]:
df = pd.get_dummies(df, columns=['mkt_op_combo'])

In [63]:
df = pd.get_dummies(df, columns=['fl_type']) 

In [65]:
df = pd.get_dummies(df, columns=['origin']) 

In [66]:
df = pd.get_dummies(df, columns=['dest']) 

---

Repeat With Full Flights Data

In [70]:
df = pd.read_csv("flights_nullsremoved.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


*Pre-Step*

In [71]:
df.drop(columns=['dep_time', 'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in','arr_time','cancellation_code', 'diverted', 'actual_elapsed_time', 'first_dep_time', 'total_add_gtime', 'longest_add_gtime'], inplace = True)

*Step 1*

In [72]:
df = df[df.arr_delay >= -225]
df = df[df.arr_delay <= 225]

*Step 2*

In [73]:
df.flights.unique()

array([1.])

In [74]:
df.dup.unique()

array(['N'], dtype=object)

*Step 3*

In [75]:
df.drop(columns = ['mkt_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'dest_airport_id', 'dup', 'flights'], inplace = True)

*Step 4*

In [76]:
#fl_month
df.loc[:,'fl_month'] = df['fl_date'].apply(get_month)

In [77]:
#fl_day_of_week
df.loc[:,'fl_day_of_week'] = df['fl_date'].apply(get_day_of_week)

In [78]:
#drop_fl_date
df.drop(columns = 'fl_date', inplace=True)

*Step 5*

In [79]:
#get mkt_op_combo
df.loc[:,"mkt_op_combo"] = df.apply(lambda x: mkt_op_combo(x.mkt_unique_carrier, x.op_unique_carrier), axis=1)

In [80]:
df.drop(columns = ['mkt_unique_carrier', 'op_unique_carrier'], inplace = True)

*Step 6*

In [81]:
df.loc[:,"branded_code_share"] = df["branded_code_share"].apply(check_codeshare)

*Step 7*

In [82]:
#get fl_type
df.loc[:,"fl_type"] = df["mkt_carrier_fl_num"].apply(flight_type)

In [83]:
df.drop(columns = 'mkt_carrier_fl_num', inplace=True)

*Step 8*

In [84]:
df.loc[:,'state_travel_type'] = df.apply(lambda x: get_staterelation(x.origin_city_name, x.dest_city_name), axis=1)

In [85]:
df.drop(columns = ['origin_city_name','dest_city_name'], inplace=True)

*Step 9*

In [86]:
df.loc[:,'crs_dep_time'] = df['crs_dep_time'].apply(check_time_block)

*Step 10*

In [87]:
df.loc[:,'crs_arr_time'] = df['crs_arr_time'].apply(check_time_block)

*Step 11*

In [88]:
df.loc[:,"origin"] = df["origin"].astype('category')
df.loc[:,"origin_cat"] = df["origin"].cat.codes

In [90]:
df.loc[:,"dest"] = df["dest"].astype('category')
df.loc[:,"dest_cat"] = df["dest"].cat.codes

In [91]:
df.loc[:,"mkt_op_combo"] = df["mkt_op_combo"].astype('category')
df.loc[:,"mkt_op_combo"] = df["mkt_op_combo"].cat.codes

In [93]:
df.drop(columns = ['origin','dest','mkt_op_combo'], inplace=True)

In [95]:
df.shape

(15768096, 19)

In [96]:
df.to_csv("flights_cleaned.csv", index = False)